In [ ]:
import pickle
import os
from tqdm.auto import tqdm
from syntheseus.search.graph.message_passing import run_message_passing
from syntheseus.search.analysis.tree_solution_counting import num_solutions_update
from syntheseus.search.analysis.route_extraction import min_cost_routes
from syntheseus.search.graph.and_or import OrNode, AndNode
from syntheseus.search.analysis.route_extraction import _iter_top_routes, _min_route_cost, _min_route_partial_cost

from syntheseus.search import visualization


## Settings

In [ ]:

# cost_type = "cost_1_react"
# cost_type = "cost_react_from_data"
# cost_type = "cost_react_from_data_pow01"

run_id = '202305-2911-2320-5a95df0e-3008-4ebe-acd8-ecb3b50607c7'
# input_file = f'Runs/{run_id}/routes_df.csv'
# routes_df = pd.read_csv(input_file)

input_folder = f'Runs/{run_id}/constant0_graph_pickles'
input_file_routes = f'Runs/{run_id}/targ_routes.pickle'
input_file_distances = f'Runs/{run_id}/targ_to_purch_distances.pickle'

# with open(input_file_routes, 'rb') as handle:
#     targ_routes_dict = pickle.load(handle)
    
# # Load distances data
# with open(input_file_distances, 'rb') as handle:
#     distances_dict = pickle.load(handle)


output_folder = f'Plots/{run_id}/Sample_routes'



In [ ]:
list_smiles = [
    "COc1c(N2CCN(C(=O)COC(C)=O)CC2)cc(Br)cc1C(C)(C)C", # Nice (1)
    "C=CC(OCOC)c1ccc(N(c2cc3oc(-c4ccc(F)cc4)c(C(=O)NC)c3cc2C2CC2)S(C)(=O)=O)cc1Cl", # Route molecole easy all'inizio, complesse in profondità
#     "CC(C(=O)O)c1ccc2nc(N)oc2c1",
#     "Cc1ccc2ccccc2c1CC(C)Oc1ccccc1N",
#     "Cc1c(Oc2ccc(C(CO)CO)cc2)ncnc1OC1CCN(c2ncccn2)CC1",
#     "CCCCCCCCCCCCCCCCSCC(CBr)OC",
    "O=C(NCCc1ccc(Cl)cc1C1CC1)c1ccc(Oc2cc3c(cc2Cl)C(C(=O)O)CCO3)cc1", # Nice (2)
    "N#Cc1ccccc1-c1cc(F)cc(-c2ccnc3nc(C(F)(F)F)ccc23)c1", # Nice (3)
    "Cc1cccc2c(Cl)ncc(C(=O)NCC3CCOCC3)c12", # Ok
#     "Cc1nc(C(C)(CS(C)(=O)=O)NC(=O)c2cc(O[C@@H](C)C(F)(F)F)c(C3CCC3)cn2)no1",
#     "CC(C)(CC1CCN(c2ccc([N+](=O)[O-])cn2)CC1)C(=O)O",
#     "FC(F)(F)c1ccc(COc2ccc(Br)cc2CC2CNC2)o1",
#     "CC(=O)NCCOc1ccc2c(c1)c(Cl)cn2S(=O)(=O)c1ccccc1",
#     "COc1ccc(Oc2ccccc2Cl)cc1C(C)C(=O)O",
#     "CC1(NC(=O)OC(C)(C)C)CCCN(c2c(N)cnc3c2CCC3)C1",
#     "CCN1CCN(C2(C(=O)N3C[C@H](S(=O)(=O)c4ccccc4Cl)C[C@H]3C(=O)NC3(C#N)CC3)CC2)CC1",
#     "Oc1ccc2nc(-c3ccc(-c4nn[nH]n4)cc3Cl)ccc2c1",
#     "CCOC(=O)Cc1ccc(OC)c(-c2ccc(NC(C)=O)cc2CN(CC)C(=O)OCc2ccccc2)c1",
#     "Cc1cccc(NC2(CO)CCN(Cc3ccccc3)CC2)c1",
    "CN(C)Cc1cc(C(C)(C)C)cc(Cl)n1", # Nice
#     "CCOC(=O)C(C)(CCCc1ccccc1)Cc1ccc(OCCN)cc1",
#     "CCCC(Oc1ccccc1Br)C(C)=O",
#     "CN(Cc1ccccc1)C(=O)c1cc(NC(=O)c2cc(F)c(F)cc2Cl)[nH]n1",
    "COC(=O)c1cc(N=Nc2ccc(OCCCCCCCCN=[N+]=[N-])cc2)ccc1O", # From 1 reaction exit 3 molecules (but target not nice, long chain)
]

## Run 

In [ ]:
cost_type = "cost_1_react"
# cost_type = "cost_react_from_data"
# cost_type = "cost_react_from_data_pow01"



def custom_assign_cost(graph, cost_type):
    if cost_type == "cost_1_react": 
        for node in graph._graph.nodes():
                if isinstance(node, (AndNode,)):
                    node.data["route_cost"] = 1.0
                else:
                    node.data["route_cost"] = 0.0
    elif cost_type == "cost_react_from_data": 
        for node in graph._graph.nodes():
                if isinstance(node, (AndNode,)):
                    node.data["route_cost"] = node.data["retro_star_rxn_cost"]
                else:
                    node.data["route_cost"] = 0.0
    elif cost_type == "cost_react_from_data_pow01": 
        for node in graph._graph.nodes():
                if isinstance(node, (AndNode,)):
                    node.data["route_cost"] = np.power(node.data["retro_star_rxn_cost"], 0.1)
                else:
                    node.data["route_cost"] = 0.0
    else:
        raise NotImplementedError(f'Cost type {cost_type}')
    
    return output_graph



In [ ]:
for file_name in tqdm([file for file in os.listdir(input_folder) if 'pickle' in file]):
    with open(f'{input_folder}/{file_name}', 'rb') as handle:
        for smiles, output_graph in (pickle.load(handle)).items():
            if smiles in list_smiles:
                output_graph = custom_assign_cost(output_graph, cost_type)
                
                for route_rank, (route_cost, route) in enumerate(_iter_top_routes(
                        graph=output_graph,
                        cost_fn=_min_route_cost,
                        cost_lower_bound=_min_route_partial_cost,
                        max_routes=1,
                        yield_partial_routes=False,
                    )):
                        # Visualise route
                        visualization.visualize_andor(
                            graph=output_graph,
                            filename=f"{output_folder}/route_{route_rank+1}_{smiles}.pdf",
                            nodes=route,
                        )
                

                visualization.visualize_andor(
                            graph=output_graph,
                            filename=f"{output_folder}/wholeGraph_{smiles}.pdf",
                        )

                        


In [ ]:
os.getcwd()